In [6]:
class Node:
    def __init__(self, x=None, y=None, next_node=None,
    i=None, j=None, g_value=0, h_value=0):
        self.data = [x, y]
        self.next_node = next_node if next_node is not None else []
        self.parent = None
        self.g_value = g_value
        self.h_value = h_value
        self.f_value = self.g_value + self.h_value

    def __lt__(self, other):
        return self.f_value < other.f_value

In [10]:
import matplotlib.pyplot as plt
import random
import numpy as np
import heapq
from queue import PriorityQueue

import time


N = 101

updatedHeuristicsMap = {}

def updatedHeuristics(finalNode):
  curr = finalNode
  while curr is not None:
    updatedHeuristicsMap[(curr.data[0],curr.data[1])] = finalNode.g_value - curr.g_value
    curr = curr.parent

def getUpdatedHeuristic(node,targetNode):
  if (node.data[0],node.data[1]) in updatedHeuristicsMap:
    return updatedHeuristicsMap[(node.data[0],node.data[1])]
  else:
    return manhattan(node,targetNode)



def manhattan(initialCell, goalCell):
    return abs(initialCell.data[0] - goalCell.data[0]) + abs(initialCell.data[1] - goalCell.data[1]) # goalCell is used twice since x and y will always be the same (size 5)

def aStar(grid, start, target): #grid must be 2d array, start and target must be 1d arrays of two elements

    startNode = Node(start[0],start[1],None,None,None,0,manhattan(Node(start[0],start[1]),Node(target[0],target[1])))
    targetNode = Node(target[0],target[1])

    openList = [] #open list
    heapq.heappush(openList,startNode)
    closed = set() #closed list
    while openList:
      currNode = heapq.heappop(openList)

      if tuple(currNode.data) in closed: # Cannot append arrays to an sets, so i made it into a tuple
        continue

      if currNode.data == targetNode.data: # Path found
        finalPath = []
        while currNode: # While a parent exists for the node
          finalPath.append(currNode.data)
          currNode = currNode.parent
        return finalPath[::1] # Returning reversed path so that the array starts at startingNode and ends with targetNode

      closed.add(tuple(currNode.data))

      for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]: # All neighbors for current cell
        x = currNode.data[0] + dx
        y = currNode.data[1] + dy

        if 0 <= x < len(grid) and 0 <= y < len(grid[0]) and grid[x][y] == 0:
          neighbor = Node(x,y,None,None,None,currNode.g_value + 1,manhattan(Node(x,y),targetNode))
          neighbor.parent = currNode

          if (x,y) not in closed:
            heapq.heappush(openList, neighbor)

    return None #if no path is found

def adaptiveA(grid, start, target):
    startNode = Node(start[0],start[1],None,None,None,0,manhattan(Node(start[0],start[1]),Node(target[0],target[1])))
    targetNode = Node(target[0],target[1])

    openList = [] #open list
    heapq.heappush(openList,startNode)
    closed = set() #closed list
    while openList:
      currNode = heapq.heappop(openList)

      if tuple(currNode.data) in closed: # Cannot append arrays to an sets, so i made it into a tuple
        continue

      if currNode.data == targetNode.data: # Path found
        updatedHeuristics(currNode) # Finds each parent for the current node (which is target) and stores their actual distance from node to target in map for possible later use
        finalPath = []
        while currNode: # While a parent exists for the node
          finalPath.append(currNode.data)
          currNode = currNode.parent
        return finalPath[::1] # Returning reversed path so that the array starts at startingNode and ends with targetNode

      closed.add(tuple(currNode.data))

      for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]: # All neighbors for current cell
        x = currNode.data[0] + dx
        y = currNode.data[1] + dy

        if 0 <= x < len(grid) and 0 <= y < len(grid[0]) and grid[x][y] == 0:
          neighbor_h_value = getUpdatedHeuristic(Node(x,y),targetNode) # Attempts to see if the previous distance has been calculated already, if not: manhattan distance, if so returns the accurate heuristic
          neighbor = Node(x,y,None,None,None,currNode.g_value + 1,neighbor_h_value)
          neighbor.parent = currNode

          if (x,y) not in closed:
            heapq.heappush(openList, neighbor)

    return None #if no path is found


###################### TESTING
start_time = time.time()


#grid = [[0,1,1,0,1],[0,0,0,0,1],[1,0,1,1,1],[1,0,0,0,0],[1,1,0,1,0]]
#startCoords = [0,0]
#endCoords = [4,4]

successes, failures = 0,0
for i in range(0,50):

  grid = [[random.choices([0, 1], weights=[0.7, 0.3], k=1)[0] for _ in range(N)] for _ in range(N)]
  startCoords = [random.randint(0, N-1),random.randint(0, N-1)]
  endCoords = [random.randint(0, N-1),random.randint(0, N-1)]

  printedPath = aStar(grid,startCoords,endCoords)

  if printedPath is not None:
    grid[startCoords[0]][startCoords[1]] = 5 #starting cell, light blue
    grid[endCoords[0]][endCoords[1]] = 9 #end cell, dark blue
    for array in printedPath: #now import dfs and check to see if the path exists and this works@
      grid[array[0]][array[1]] = 7
    successes += 1
  else:
    failures += 1

#image_data = np.array(grid)
#plt.imshow(grid, "Blues")
#plt.show()

# Run the algorithm
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

print(f"success: {successes}, failures: {failures}, Chance of path creation: {successes/100}")

<class '__main__.Node'>
HELLO


AttributeError: 'list' object has no attribute 'length'